In [1]:
%matplotlib inline
import matplotlib as mpl
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import minmax_scale
from sklearn.decomposition import PCA


In [2]:
#carrega o dataset
dados = pd.read_csv('/home/erasmor/Área de Trabalho/CSE-CIC-IDS2018/amostras_estratificadas/1amostragem_todas_instancias_somadas_baixa_alta_representatividade_estratificada.csv',sep=",",encoding = 'utf-8',  header=0,na_values='.')

#remove valores infinitos
dados.replace(-np.Inf, np.nan)

#substitui valores NaN
dados.fillna(dados.mean())


,Dst Port,Protocol,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,53,17,886,1,1,38,86,38,38,38.000000,...,8,0.0,0.0,0,0,0.0,0.0,0.0,0.0,Infilteration
1,53,17,1079,1,1,28,78,28,28,28.000000,...,8,0.0,0.0,0,0,0.0,0.0,0.0,0.0,Infilteration
2,445,6,262998,3,1,0,0,0,0,0.000000,...,20,0.0,0.0,0,0,0.0,0.0,0.0,0.0,Infilteration
3,3389,6,1824650,8,7,1128,1581,661,0,141.000000,...,20,0.0,0.0,0,0,0.0,0.0,0.0,0.0,Infilteration
4,53,17,1102,1,1,49,77,49,49,49.000000,...,8,0.0,0.0,0,0,0.0,0.0,0.0,0.0,Infilteration
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8467,3389,6,3868068,14,8,1431,1727,725,0,102.214286,...,20,0.0,0.0,0,0,0.0,0.0,0.0,0.0,Benign
8468,3389,6,3996691,15,8,1434,1727,725,0,95.600000,...,20,0.0,0.0,0,0,0.0,0.0,0.0,0.0,Benign
8469,3389,6,3945545,14,8,1451,1727,741,0,103.642857,...,20,0.0,0.0,0,0,0.0,0.0,0.0,0.0,Benign
8470,3389,6,3972090,14,8,1432,1727,725,0,102.285714,...,20,0.0,0.0,0,0,0.0,0.0,0.0,0.0,Benign


In [4]:
#mostra como está a base de dados
#dados.head()

In [5]:
# verifica quantas instâncias (linhas) e quantos atributos (colunas) a base de dados contém
print("numero de linhas e colunas: ",dados.shape)

numero de linhas e colunas:  (8472, 79)


In [6]:
#visualizar distribições por classes contidas no csv - informar nome da classe alvo
print(dados.groupby('Label').size())

Label
Benign                      1870
Bot                          520
Brute Force -Web             611
Brute Force -XSS             230
DDOS attack-HOIC            1248
DDOS attack-LOIC-UDP        1730
DoS attacks-GoldenEye         75
DoS attacks-Hulk             840
DoS attacks-SlowHTTPTest     254
DoS attacks-Slowloris         20
FTP-BruteForce               352
Infilteration                294
SQL Injection                 87
SSH-Bruteforce               341
dtype: int64


In [ ]:
#X_raw_normalize = MinMaxScaler(X_raw_normalize.reshape(0, 1)).reshape(len(X_raw_normalize))
#X_raw_normalizetd2 = (X_raw_normalize - X_raw_normalize.min(axis=0)) / (X_raw_normalize.max(axis=0) - X_raw_normalize.min(axis=0))
# Obtendo os nomes das colunas do DataFrame como uma lista.
cols = list(dados.columns)
# colunas que nao serao normalizadas
cols.remove('Label')

# Copiando os dados e aplicando a normalizacao por reescala nas colunas do DataFrame que contem
# valores continuos. Por padrao, o metodo minmax_scale reescala com min=0 e max=1.
dados = dados[~dados.isin([np.nan, np.inf, -np.inf]).any(1)]
dados[cols] = dados[cols].apply(minmax_scale)


In [ ]:
#define as colunas de atributos e a coluna da classe (de 0 a 72 são atributos e após a 72 é a classe)
# "X_raw" é features/atributos e "y_raw" é target/classe ==> As duas formas abaixo dão certo.
#array = dataset.values
#X_raw = array[:,0:72]
#y_raw = array[:,72]
X_raw = dados.iloc[:, :-1].values # atributos
y_raw = dados.iloc[:, 78].values # classe de ataques
X_raw = np.nan_to_num(X_raw.astype(np.float32))


In [ ]:
#transformar a variável Y com valores categóricos das classses de ataques em valores numéricos:
labelencoder_y = LabelEncoder()
y_raw = labelencoder_y.fit_transform(y_raw)


In [ ]:
# Instanciando um PCA. O parametro n_components indica a quantidade de dimensoes que a base
# original sera reduzida.
pca = PCA(n_components=10, whiten=True,random_state=0)

In [ ]:
# Aplicando o pca na base de dados. O atributo 'values' retorna um numpy.array
# de duas dimensões (matriz) contendo apenas os valores numericos do DataFrame.
X_raw = pca.fit_transform(X_raw)

In [ ]:
def uncertain_sampling(X_raw, y_raw, idx_data, idx_dobra, TRAIN, TEST, classifier, t_inicial):
    
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.naive_bayes import GaussianNB
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.svm import LinearSVC
    from sklearn.metrics import precision_score, recall_score, confusion_matrix, classification_report, accuracy_score, f1_score
    import functools
    from IPython.display import clear_output
    
    #define nome de arquivos para salvar
    indica_pool=str(idx_dobra)
    arquivo_performance_knn = ("uncertain_performance_knn_dobra_"+indica_pool+".csv")
    arquivo_performance_rf = ("uncertain_performance_rf_dobra_"+indica_pool+".csv")
    arquivo_performance_nb = ("uncertain_performance_nb_dobra_"+indica_pool+".csv")
    arquivo_performance_tree = ("uncertain_performance_tree_dobra_"+indica_pool+".csv")
    arquivo_performance_mlp = ("uncertain_performance_mlp_dobra_"+indica_pool+".csv")
    
    # recupera as amostras de treino iniciais - a extratificação realizada só serve para tal finalidade.
    # No caso força a buscar pelo menos uma amostras de cada rótulo disponível (train_size= len(np.unique(y_raw)).
    # Realizar a busca aleatoriamente não garante iniciar com uma instância de cada classe.
    X_train, X_test, y_train, y_test = train_test_split(X_raw[idx_data[idx_dobra][TRAIN]], y_raw[idx_data[idx_dobra][TRAIN]], train_size= len(np.unique(y_raw)) + t_inicial, stratify = y_raw[idx_data[idx_dobra][TRAIN]])
    #print("tamanho de X_train inicial: ",X_train.shape)
    amostras_iniciais=X_train.shape
    n_amostras_iniciais = int(amostras_iniciais[0])
    #define array de performance
    performance_history = []
    #instanciando classificadores de aprendizado ativo
    if classifier == "knn":
        learner = ActiveLearner(estimator=KNeighborsClassifier(n_neighbors=3),X_training=X_train, y_training=y_train)
        arquivo_performance = open("uncertain_performance_knn_dobra_"+indica_pool+".txt","a")
        arquivo_history = ("uncertain_history_knn_dobra_"+indica_pool+".csv")
        titulo_grafico = 'Incremental classification accuracy KNN'
                
    if classifier == "rf":
        learner = ActiveLearner(estimator=RandomForestClassifier(max_depth=2),X_training=X_train, y_training=y_train)
        arquivo_performance = open("uncertain_performance_rf_dobra_"+indica_pool+".txt","a")
        arquivo_history = ("uncertain_history_rf_dobra_"+indica_pool+".csv")
        titulo_grafico = 'Incremental classification accuracy RF'
        
        
    if classifier == "nb":
        learner = ActiveLearner(estimator=GaussianNB(),X_training=X_train, y_training=y_train)
        arquivo_performance = open("uncertain_performance_nb_dobra_"+indica_pool+".txt","a")
        arquivo_history = ("uncertain_history_nb_dobra_"+indica_pool+".csv")
        titulo_grafico = 'Incremental classification accuracy NB'
      
        
    if classifier == "tree":
        learner = ActiveLearner(estimator=DecisionTreeClassifier(),X_training=X_train, y_training=y_train)
        arquivo_performance = open("uncertain_performance_tree_dobra_"+indica_pool+".txt","a")
        arquivo_history = ("uncertain_history_tree_dobra_"+indica_pool+".csv")
        titulo_grafico = 'Incremental classification accuracy TREE'
       
        
    
    # recupera amostras de teste de acordo com a dobra em uso
    X_test, y_test = X_raw[idx_data[idx_dobra][TEST]], y_raw[idx_data[idx_dobra][TEST]]
    # recupera amostras de treino de acordo com a dobra em uso
    X_train, y_train = X_raw[idx_data[idx_dobra][TRAIN]], y_raw[idx_data[idx_dobra][TRAIN]]
    
    ##Registro da pontuação na porção de teste com o treinamento inicial
    uncertain_sample_score = learner.score(X_test, y_test)
    performance_history.append(uncertain_sample_score)
    
    #define o pool
    X_pool = X_train
    y_pool = y_train
    
    #verifica tamanho do conjunto de treino para indicar o numero de amostras disponiveis na dobra
    print("tamanho do pool: ",X_pool.shape)
    amostras_pool=X_pool.shape
    n_amostras_pool = int(amostras_pool[0])
    print(n_amostras_pool)    
    #inicio aprendizado ativo
     
    for index in range(n_amostras_pool):
        #recupera amostras do pool baseado na estratégia de consulta
        query_index, query_instance = learner.query(X_pool)
        
        # Ensina ao modelo ActiveLearner o registro solicitado (amostras vão para o topo).
        learner.teach(X=X_pool[query_index].reshape(1, -1),y=y_pool[query_index].reshape(1, ))
       
        # apaga os registros consultados evitando que seja reconsultados
        X_pool = np.delete(X_pool, query_index, axis=0)
        y_pool = np.delete(y_pool, query_index)
        #print("tamanho de X_pool está diminuindo: ",X_pool.shape)
        # verifica a performance no conjunto de validação visto que o modelo foi treinado com novos dados
        uncertain_sample_score = learner.score(X_test, y_test)
        predictions = learner.predict(X_test)
        #print ('Accuracy after query no. %d: %f' % (index+1, uncertain_sample_score))
        arquivo_performance.write('Accuracy after query no. %d: %f \n' % (index+1,uncertain_sample_score))
        performance_history.append(uncertain_sample_score)
        #print ('Precision after query no. %d: %f' % (index+1, precision_score(y_test, predictions,average='macro',zero_division=1)))
        arquivo_performance.write('Precision after query no. %d: %f \n' % (index+1,precision_score(y_test, predictions,average='macro',zero_division=1)))
        #print ('Recall after query no. %d: %f' % (index+1, recall_score(y_test, predictions, average='macro',zero_division=1)))
        arquivo_performance.write('Recall after query no. %d: %f \n' % (index+1, recall_score(y_test, predictions, average='macro',zero_division=1)))
        #print ('F1 score after query no. %d: %f' % (index+1, f1_score(y_test, predictions,average='macro',zero_division=1)))
        arquivo_performance.write('F1 Score after query no. %d: %f \n' % (index+1, f1_score(y_test, predictions,average='macro',zero_division=1)))
        f1score= 2*((precision_score(y_test, predictions,average='macro',zero_division=1)*recall_score(y_test, predictions, average='macro',zero_division=1))/(precision_score(y_test, predictions,average='macro',zero_division=1)+recall_score(y_test, predictions, average='macro',zero_division=1)))
        arquivo_performance.write('F1 score calculado after query no. %d: %f \n' % (index+1, f1score))
        #print ("========================================")
        arquivo_performance.write('======================================== \n')
        for i in range(2):
            clear_output(wait=True)
            print(i,"Aguarde, em execução!", flush=True)                
        
      
    np.savetxt(arquivo_history, performance_history,delimiter=",")
    arquivo_performance.close()
    
     

import time
import sys
import threading
from datetime import datetime
from datetime import date

from sklearn.model_selection import StratifiedShuffleSplit, ShuffleSplit, train_test_split
from modAL.uncertainty import classifier_uncertainty
from modAL.models import ActiveLearner
from modAL.uncertainty import uncertainty_sampling
from timeit import Timer
import time
import functools
# importing the multiprocessing module
import multiprocessing

#inicia relogio
t1 = time.time()
# Define o tamanho das divisões feitas no dataset (cross-validation)
n_dobras = 10
# Define Tamanho inicial da amostra (toda estratégia parte de um tamanho mínimo).
t_inicial = 10
#define array de performance
performance_history = []
#define array de indices das partições
idx_data =[]
# cross validation bags - n_splits
data_cv = StratifiedShuffleSplit(n_splits= n_dobras, random_state=0) 
data_cv.get_n_splits(X_raw, y_raw)
# chame a instância e gere os dados sobre a base original
type(data_cv.split(X_raw, y_raw))
# dividir os dados - A função split.split () retorna índices para amostras de treino e amostras de teste. 
# Ele examinará o número de validação cruzada especificado e retornará cada vez que treinar 
# e testar os índices de amostra usando os conjuntos de dados de treinamento e teste que podem 
# ser criados filtrando o conjunto de dados inteiro. Por exemplo idx_data[0][1], o primeiro indice faz referencia
# a dobra e o segundo indice faz referencia a posição da dobra (0 = treino e 1 = teste). Logo TRAIN=0 e TEST=1.
for train_index, test_index in data_cv.split(X_raw,y_raw):
    #print("TRAIN:", train_index, "TEST:", test_index)
    #print("n_split",n_splits,"TRAIN:", train_index, "TEST:", test_index)
    idx_data.append([train_index, test_index])
#verifica tamanho das dobras (numero de instâncias de cada dobra)
#print("tamanho de cada dobra: ",idx_data[3][0].shape)

TRAIN =0
TEST =1
#chama procedimento de aprendizado para todas as dobras
#classificadores=['knn'] 
classificador_knn="knn"
classificador_rf="rf"
classificador_nb="nb"
classificador_tree="tree"
#classificadores=['knn','rf','nb','tree']
#for classificador in classificadores:
#    classifier = classificador
#    for idx_dobra in range(n_splits):
        #uncertain_sampling(X_raw, y_raw, idx_data, idx_dobra, TRAIN, TEST, classifier, init_size)
for idx_dobra in range(n_dobras):
    if __name__ == "__main__":
        # criando os processos
        p1 = multiprocessing.Process(target=uncertain_sampling(X_raw, y_raw, idx_data, idx_dobra, TRAIN, TEST, classificador_knn, t_inicial))
        p2 = multiprocessing.Process(target=uncertain_sampling(X_raw, y_raw, idx_data, idx_dobra, TRAIN, TEST, classificador_rf, t_inicial))
        p3 = multiprocessing.Process(target=uncertain_sampling(X_raw, y_raw, idx_data, idx_dobra, TRAIN, TEST, classificador_nb, t_inicial))
        p4 = multiprocessing.Process(target=uncertain_sampling(X_raw, y_raw, idx_data, idx_dobra, TRAIN, TEST, classificador_tree, t_inicial))
        # iniciando os processos
        p1.start()
        p2.start()
        p3.start()
        p4.start()
        
        # aguardando os processos serem finalizados
        p1.join()
        p2.join()
        p3.join()
        p4.join()
        
        # todos os processos finalizados
        print("Terminado!")
 
t2 = time.time()
time_elapsed = (t2-t1)/3600
print("tempo: ",time_elapsed)
